https://www.kaggle.com/ryanxjhan/cbc-news-coronavirus-articles-march-26

http://openaccess.uoc.edu/webapps/o2/bitstream/10609/81435/6/jsobrinosTFM0618memoria.pdf

In [22]:
import nltk
from gensim import models
import pandas 
import json
import numpy
import requests
from random import randint, uniform,random

#Análisis de sentimientos
from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer



#Traductor
from nltk.translate import AlignedSent, Alignment

import matplotlib.pyplot as plt

##### Editamos el LEXICON al lexicon que viene integrado en SentimentalIntensityAnalizer 

In [2]:
lexicon = pandas.read_csv('SEL/SEL_UTF8.txt', sep='\t', names=['Palabra', 'PFA', 'Categoría'])
lexicon.drop(['Categoría'], axis='columns', inplace=True)
lexicon.info()

Palabras = numpy.array(lexicon.Palabra)
PFA = numpy.array(lexicon.PFA)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2037 entries, 0 to 2036
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Palabra  2037 non-null   object
 1   PFA      2037 non-null   object
dtypes: object(2)
memory usage: 32.0+ KB


In [3]:
lista = []
for i in range(1, len(Palabras)):
    lista.append([Palabras[i], float(PFA[i])])
    
lexi = {a : b for a,b in lista}
lexi

{'abundancia': 0.83,
 'acabalar': 0.396,
 'acallar': 0.198,
 'acatar': 0.198,
 'acción': 0.397,
 'aceptable': 0.594,
 'aceptación': 0.696,
 'acicate': 0.429,
 'aclamación': 0.799,
 'aclamar': 0.799,
 'acogedor': 0.83,
 'acoger': 0.729,
 'acomodadamente': 0.729,
 'acuciar': 0.264,
 'acucioso': 0.232,
 'adecuar': 0.331,
 'adicto': 0.429,
 'admirable': 0.73,
 'admirablemente': 0.663,
 'admiración': 0.764,
 'admirar': 0.73,
 'admitir': 0.53,
 'adorable': 0.898,
 'adorablemente': 0.865,
 'adoración': 0.765,
 'adorador': 0.664,
 'adorar': 0.764,
 'afable': 0.696,
 'afán': 0.764,
 'afectivo': 0.864,
 'afecto': 0.899,
 'afectuosamente': 0.966,
 'afectuosidad': 0.932,
 'afectuoso': 0.898,
 'afervorizar': 0.731,
 'afición': 0.729,
 'aficionar': 0.595,
 'afortunadamente': 0.831,
 'afortunado': 0.932,
 'agradable': 0.899,
 'agradar': 0.798,
 'agradecer': 0.763,
 'agradecido': 0.764,
 'agudeza': 0.331,
 'aguzar': 0.199,
 'ahínco': 0.73,
 'ahíto': 0.564,
 'airoso': 0.565,
 'alabar': 0.53,
 'alboroza

##### Integramos el nuevo Lexicon al lexicon base 

In [4]:
SIA = SentimentIntensityAnalyzer()
SIA.lexicon.update(lexi)

Librería vaderSentiment
* neu=neutral
* compound= es una regla que normaliza, digamos que puede ser parecido a una media de los otros valores
* pos=positivo
* neg=negativo

En general podemos decir:

* positivo sentimiento: compound >= 0.5
* neutral sentimiento: (compound > -0.5) and (compound < 0.5)
* negativo sentimiento: compound <= -0.5

In [110]:
def ClasificadorSentimientos(dataFrame):
    arrAux = numpy.array(dataFrame.Valor)
    indice = numpy.where(arrAux == max(arrAux))[0][0]

    
    if indice == 0:
        print("Negativo")
    elif indice == 1:
        print("Neutro")
    elif indice == 2:
        print("Positivo")

#### Verificamos la precisión del analizador de sentimientos con el nuevo lexicon

In [20]:
tweets = pandas.read_csv("TweetsClasificados.csv", names=['Tweet', 'Polaridad'])
tweets =tweets.dropna()
tweets.info()
arrayTweets = tweets.to_numpy()
print(arrayTweets.shape)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 989 entries, 0 to 1976
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Tweet      989 non-null    object
 1   Polaridad  989 non-null    object
dtypes: object(2)
memory usage: 23.2+ KB
(989, 2)


In [39]:
tw = numpy.asarray(tweets.Tweet)
tw[0]

"— ¿Nivel de inglés? — Alto. — Traduzca juguete. — Toy. — Úselo en una oración. — Toy triste. — Awww, vengache pa' acá. —"

In [170]:
Pos = 0
Neg = 0
Neu = 0
for i in range(len(tw)):
    tweet = tw[i]
    vs = SIA.polarity_scores(tweet)
    auxArr = numpy.split(numpy.asarray(list(vs.items())).T, 2)
    #polarity = numpy.asarray(auxArr[0], dtype =str).flatten()
    values = numpy.asarray(auxArr[1], dtype =float).flatten()
    compound = values[3]
    print(compound)
    if (compound > 0.3):
        print("Positivo -> Valor = ", values)
        Pos += 1
    elif (compound <= 0.3 and compound > -0.3):
        print("Negativo -> Valor = ", values)
        Neg += 1
    elif(compound <= 0.3):
        print("Neutro -> Valor = ", values)
        Neu += 1
    #print("Sentimiento = ", polarity[indice], "Valor = ", values)

0.242
Negativo -> Valor =  [0.    0.924 0.076 0.242]
0.0
Negativo -> Valor =  [0. 1. 0. 0.]
0.6649
Positivo -> Valor =  [0.     0.837  0.163  0.6649]
-0.3442
Neutro -> Valor =  [ 0.102   0.898   0.     -0.3442]
-0.1791
Negativo -> Valor =  [ 0.142   0.708   0.151  -0.1791]
0.4547
Positivo -> Valor =  [0.     0.863  0.137  0.4547]
-0.7172
Neutro -> Valor =  [ 0.253   0.705   0.042  -0.7172]
0.0418
Negativo -> Valor =  [0.07   0.824  0.107  0.0418]
-0.3298
Neutro -> Valor =  [ 0.099   0.851   0.051  -0.3298]
0.1464
Negativo -> Valor =  [0.     0.927  0.073  0.1464]
0.0
Negativo -> Valor =  [0. 1. 0. 0.]
-0.296
Negativo -> Valor =  [ 0.075  0.925  0.    -0.296]
-0.4215
Neutro -> Valor =  [ 0.116   0.884   0.     -0.4215]
0.7063
Positivo -> Valor =  [0.     0.785  0.215  0.7063]
0.0
Negativo -> Valor =  [0. 1. 0. 0.]
0.0
Negativo -> Valor =  [0. 1. 0. 0.]
-0.296
Negativo -> Valor =  [ 0.071  0.929  0.    -0.296]
-0.296
Negativo -> Valor =  [ 0.066  0.934  0.    -0.296]
-0.296
Negativo -> V

0.4826
Positivo -> Valor =  [0.     0.88   0.12   0.4826]
-0.3335
Neutro -> Valor =  [ 0.171   0.704   0.125  -0.3335]
-0.296
Negativo -> Valor =  [ 0.078  0.922  0.    -0.296]
-0.6808
Neutro -> Valor =  [ 0.223   0.777   0.     -0.6808]
0.4023
Positivo -> Valor =  [0.     0.829  0.171  0.4023]
0.032
Negativo -> Valor =  [0.09  0.775 0.136 0.032]
0.2018
Negativo -> Valor =  [0.     0.914  0.086  0.2018]
0.4826
Positivo -> Valor =  [0.     0.87   0.13   0.4826]
0.4585
Positivo -> Valor =  [0.     0.833  0.167  0.4585]
0.3089
Positivo -> Valor =  [0.     0.917  0.083  0.3089]
0.2533
Negativo -> Valor =  [0.     0.894  0.106  0.2533]
0.4207
Positivo -> Valor =  [0.     0.826  0.174  0.4207]
0.242
Negativo -> Valor =  [0.    0.927 0.073 0.242]
0.4207
Positivo -> Valor =  [0.     0.858  0.142  0.4207]
0.1103
Negativo -> Valor =  [0.     0.941  0.059  0.1103]
0.0
Negativo -> Valor =  [0. 1. 0. 0.]
0.0
Negativo -> Valor =  [0. 1. 0. 0.]
0.0
Negativo -> Valor =  [0. 1. 0. 0.]
0.2088
Negativo -

-0.2124
Negativo -> Valor =  [ 0.155   0.737   0.108  -0.2124]
0.0849
Negativo -> Valor =  [0.     0.935  0.065  0.0849]
0.0849
Negativo -> Valor =  [0.     0.927  0.073  0.0849]
0.0
Negativo -> Valor =  [0. 1. 0. 0.]
0.1586
Negativo -> Valor =  [0.     0.917  0.083  0.1586]
0.4462
Positivo -> Valor =  [0.     0.853  0.147  0.4462]
0.0
Negativo -> Valor =  [0. 1. 0. 0.]
0.0
Negativo -> Valor =  [0. 1. 0. 0.]
0.0
Negativo -> Valor =  [0. 1. 0. 0.]
0.659
Positivo -> Valor =  [0.    0.845 0.155 0.659]
-0.296
Negativo -> Valor =  [ 0.115  0.885  0.    -0.296]
0.242
Negativo -> Valor =  [0.    0.781 0.219 0.242]
0.4114
Positivo -> Valor =  [0.     0.776  0.224  0.4114]
-0.296
Negativo -> Valor =  [ 0.239  0.761  0.    -0.296]
0.0
Negativo -> Valor =  [0. 1. 0. 0.]
0.3716
Positivo -> Valor =  [0.     0.702  0.298  0.3716]
0.0
Negativo -> Valor =  [0. 1. 0. 0.]
0.0
Negativo -> Valor =  [0. 1. 0. 0.]
-0.0603
Negativo -> Valor =  [ 0.087   0.834   0.078  -0.0603]
0.6479
Positivo -> Valor =  [0.

0.3089
Positivo -> Valor =  [0.     0.799  0.201  0.3089]
0.0
Negativo -> Valor =  [0. 1. 0. 0.]
-0.296
Negativo -> Valor =  [ 0.121  0.879  0.    -0.296]
0.0765
Negativo -> Valor =  [0.     0.925  0.075  0.0765]
-0.296
Negativo -> Valor =  [ 0.196  0.804  0.    -0.296]
0.3716
Positivo -> Valor =  [0.     0.855  0.145  0.3716]
0.2018
Negativo -> Valor =  [0.     0.87   0.13   0.2018]
-0.5574
Neutro -> Valor =  [ 0.419   0.581   0.     -0.5574]
-0.296
Negativo -> Valor =  [ 0.145  0.855  0.    -0.296]
0.0
Negativo -> Valor =  [0. 1. 0. 0.]
0.0
Negativo -> Valor =  [0. 1. 0. 0.]
-0.296
Negativo -> Valor =  [ 0.167  0.833  0.    -0.296]
0.3597
Positivo -> Valor =  [0.     0.811  0.189  0.3597]
0.0
Negativo -> Valor =  [0. 1. 0. 0.]
-0.1346
Negativo -> Valor =  [ 0.172   0.692   0.136  -0.1346]
0.1606
Negativo -> Valor =  [0.     0.908  0.092  0.1606]
0.0
Negativo -> Valor =  [0. 1. 0. 0.]
0.0
Negativo -> Valor =  [0. 1. 0. 0.]
-0.296
Negativo -> Valor =  [ 0.109  0.891  0.    -0.296]
-0.3

Negativo -> Valor =  [ 0.181   0.658   0.162  -0.0603]
-0.2271
Negativo -> Valor =  [ 0.232   0.632   0.137  -0.2271]
-0.1187
Negativo -> Valor =  [ 0.236   0.552   0.212  -0.1187]
0.0
Negativo -> Valor =  [0. 1. 0. 0.]
0.0
Negativo -> Valor =  [0. 1. 0. 0.]
0.0
Negativo -> Valor =  [0. 1. 0. 0.]
0.0
Negativo -> Valor =  [0. 1. 0. 0.]
0.127
Negativo -> Valor =  [0.    0.87  0.13  0.127]
0.0
Negativo -> Valor =  [0. 1. 0. 0.]
0.0
Negativo -> Valor =  [0. 1. 0. 0.]
0.0765
Negativo -> Valor =  [0.     0.92   0.08   0.0765]
0.0
Negativo -> Valor =  [0. 1. 0. 0.]
-0.296
Negativo -> Valor =  [ 0.104  0.896  0.    -0.296]
0.0
Negativo -> Valor =  [0. 1. 0. 0.]
0.1187
Negativo -> Valor =  [0.     0.899  0.101  0.1187]
0.0
Negativo -> Valor =  [0. 1. 0. 0.]
0.3459
Positivo -> Valor =  [0.     0.724  0.276  0.3459]
0.242
Negativo -> Valor =  [0.    0.803 0.197 0.242]
0.0
Negativo -> Valor =  [0. 1. 0. 0.]
0.0
Negativo -> Valor =  [0. 1. 0. 0.]
-0.3595
Neutro -> Valor =  [ 0.172   0.828   0.     

In [171]:
print("Positivos = ", Pos, "Negativos = ", Neg, "Neutros = ", Neu)

Positivos =  117 Negativos =  766 Neutros =  106


In [129]:
vs = SIA.polarity_scores(":)")
print(vs)

{'neg': 0.0, 'neu': 0.0, 'pos': 1.0, 'compound': 0.4588}


In [136]:
dataVS = numpy.asarray((list(vs.items()))).T
split = numpy.split(dataVS, 2)
split
#print(numpy.where(max(dataVS[1])))

#print(dataVS[0][])
#indice = numpy.where(dataVS[1] == max(dataVS))
#an_array = np. array(data)
#print(an_array)

[array([['neg', 'neu', 'pos', 'compound']], dtype='<U8'),
 array([['0.0', '0.645', '0.355', '0.4318']], dtype='<U8')]

In [116]:
df = pandas.DataFrame(vs.items(), columns=['Sentimiento', 'Valor'])
df.Sentimiento

0         neg
1         neu
2         pos
3    compound
Name: Sentimiento, dtype: object

In [117]:
ClasificadorSentimientos(df)

Positivo
